<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Нужно обучить модель классифицировать комментарии на позитивные и негативные. В распоряжении есть набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

In [30]:
import warnings
warnings.filterwarnings('ignore')
from time import time

import pandas as pd
import numpy as np
import re
import nltk
#from nltk.stem import WordNetLemmatizer
#from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import spacy
from tqdm import tqdm
tqdm.pandas()
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score

In [16]:
pd.set_option('display.max_colwidth', None)
nltk.download('stopwords')
nltk.download('punkt')  
nltk.download('wordnet')
nlp = spacy.load('en_core_web_sm')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Подготовка

In [19]:
comments_df = pd.read_csv('/datasets/toxic_comments.csv')
comments_df.info()
comments_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


,Unnamed: 0,text,toxic
0,0,"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",0
1,1,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, January 11, 2016 (UTC)",0
2,2,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",0
3,3,"""\nMore\nI can't make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of """"types of accidents"""" -I think the references may need tidying so that they are all in the exact same format ie date format etc. I can do that later on, if no-one else does first - if you have any preferences for formatting style on references or want to do it yourself please let me know.\n\nThere appears to be a backlog on articles for review so I guess there may be a delay until a reviewer turns up. It's listed in the relevant form eg Wikipedia:Good_article_nominations#Transport """,0
4,4,"You, sir, are my hero. Any chance you remember what page that's on?",0


In [20]:
comments_df['toxic'].value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

"Токсичных" комментариев почти в 10 раз меньше

In [21]:
#не уверен, что английский язык нуждается в лемматизации...
#создадим функцию лемматизации и удаления избыточных символов
'''
def clear_text(text):
    text = " ".join(re.sub(r'[^a-zA-Z\']', ' ', text).split())
    lemma = WordNetLemmatizer()
    words = word_tokenize(text)
    lemm_list = [lemma.lemmatize(word) for word in words]
    lemm_text = " ".join(lemm_list)
    return lemm_text
'''

def clear_text(text):
    text = " ".join(re.sub(r'[^a-zA-Z\']', ' ', text).split())
    doc = nlp(text)
    lemm_text = " ".join([token.lemma_ for token in doc])
    return lemm_text


In [22]:
print('Исходный текст:\n', comments_df['text'][0])
print()
print('Текст после лемматизации и обработки:\n', clear_text(comments_df['text'][0]))

Исходный текст:
 Explanation
Why the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27

Текст после лемматизации и обработки:
 explanation why the edit make under my username Hardcore Metallica Fan be revert they be not vandalism just closure on some gas after I vote at New York Dolls FAC and please do not remove the template from the talk page since I be retire now


In [44]:
sentence1 = "The striped bats are hanging on their feet for best"
sentence2 = "you should be ashamed of yourself went worked"
df_my = pd.DataFrame([sentence1, sentence2], columns = ['text'])
print(df_my)


print(df_my['text'].apply(clear_text))

                                                  text
0  The striped bats are hanging on their feet for best
1        you should be ashamed of yourself went worked
0    the stripe bat be hang on their foot for good
1        you should be ashamed of yourself go work
Name: text, dtype: object


In [26]:
comments_df['text'] = comments_df['text'].progress_apply(clear_text)

100%|██████████| 159292/159292 [32:56<00:00, 80.59it/s] 


In [28]:
#разделим выборку на тренировочную и тестовую
RANDOM_STATE = 42
TEST_SIZE = 0.25
X = comments_df['text']
y = comments_df['toxic']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=y)

In [29]:
#инициализируем счетчик с указанием стоп-слов
stopwords = set(stopwords.words('english'))

count_tf_idf = TfidfVectorizer(stop_words=stopwords)

На этапе подготовки данных загрузили данные, создали функцию лемматизации и очистки текста от ненужных символов, применили функцию к корпусу текстов, разделили выборку на тренировочную и тестовую

## Обучение

In [49]:
def ml_search(model, params):
    pipe = Pipeline(steps=[
        ('tfidf', count_tf_idf),
        ('model', model)
    ])
    rnd = RandomizedSearchCV(pipe, param_distributions=params, scoring='f1', n_jobs=-1)
    rnd.fit(X_train, y_train)
    print('Лучший результат:', rnd.best_score_)
    print('Лучшие параметры:', rnd.best_params_)
    return rnd

In [50]:
start = time()
logistic = ml_search(
    LogisticRegression(),
    {
        'model__C': [5, 10, 15],
        'model__penalty': ['none', 'l2']
    }
)
end = time()
print(f'Время выполнения функции {(end - start)/60} мин')

Лучший результат: 0.7764318994886119
Лучшие параметры: {'model__penalty': 'l2', 'model__C': 15}
Время выполнения функции 20.66081416606903 мин


In [51]:
start = time()
cat = ml_search(
    CatBoostClassifier(verbose=False),
    {
        'model__iterations': [10, 50],
        'model__learning_rate': [0.03, 0.1],
        'model__depth': [4, 6]
    }
)
end = time()
print(f'Время выполнения функции {(end - start)/60} мин')

Лучший результат: 0.6078873715761742
Лучшие параметры: {'model__learning_rate': 0.1, 'model__iterations': 50, 'model__depth': 6}
Время выполнения функции 53.19417901039124 мин


Проверим модель LogisticRegression на тестовой выборке

In [56]:
result = f1_score(y_test, logistic.predict(X_test))
print(f'Метрика f1 на лучшей модели: {result}')

Метрика f1 на лучшей модели: 0.7777169449767314


На этапе обучения для удобства создали функцию для перебора заданных гиперпараметров конкретной модели с выводом значимой информации об обученной модели. Обучили, подобрали гиперпараметры и оценили метрику f1 двух моделей LogisticRegression и CatBoostClassifier. Лучшей оказалась модель LogisticRegression. Оценили метрику лучшей модели на тестовой выборке. Полученное значение удовлетворяет заданному условию > 0.75.

## Выводы

На этапе подготовки данных загрузили данные, создали функцию лемматизации и очистки текста от ненужных символов, применили функцию к корпусу текстов, разделили выборку на тренировочную и тестовую  
На этапе обучения для удобства создали функцию для перебора заданных гиперпараметров конкретной модели с выводом значимой информации об обученной модели. Обучили, подобрали гиперпараметры и оценили метрику f1 двух моделей LogisticRegression и CatBoostClassifier. Лучшей оказалась модель LogisticRegression. Оценили метрику лучшей модели на тестовой выборке. Полученное значение удовлетворяет заданному условию > 0.75.  
Лучшая модель LogisticRegression с гиперпараметрами {'model__penalty': 'l2', 'model__C': 15} и f1 метрикой на тестовой выборке: 0.77